<a href="https://colab.research.google.com/github/sky16k/NLP/blob/main/NLP02_classif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предварительно про PyTorch:
* [Про тензоры в pytorch](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/tensor_tutorial.ipynb)
* [Про автоматическое дифференцирование и что такое .backwards()](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/autograd_tutorial.ipynb)
* [Очень простая нейронка на pytorch](https://colab.research.google.com/drive/1RsZvw4KBGn5U5Aj5Ak7OG2pHx6z1OSlF)

# Классификация текстов

## Fakenews

1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Векторизуйте.
4. Обучите на полученных векторах алгоритм классификации.

Мы уже видели как эта задача выполняется с помощью Word2vec. Давайте вспомним.

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2023-08-31 05:55:17--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2023-08-31 05:55:18 (21.9 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:07<00:00, 860.15it/s]


In [ ]:
# sentences = [word_tokenize(df.tweet[0])]
sentences

[['the',
  'cdc',
  'currently',
  'reports',
  '99031',
  'deaths',
  '.',
  'in',
  'general',
  'the',
  'discrepancies',
  'in',
  'death',
  'counts',
  'between',
  'different',
  'sources',
  'are',
  'small',
  'and',
  'explicable',
  '.',
  'the',
  'death',
  'toll',
  'stands',
  'at',
  'roughly',
  '100000',
  'people',
  'today',
  '.'],
 ['states',
  'reported',
  '1121',
  'deaths',
  'a',
  'small',
  'rise',
  'from',
  'last',
  'tuesday',
  '.',
  'southern',
  'states',
  'reported',
  '640',
  'of',
  'those',
  'deaths',
  '.',
  'https',
  ':',
  '//t.co/yasgrtt4ux'],
 ['politically',
  'correct',
  'woman',
  '(',
  'almost',
  ')',
  'uses',
  'pandemic',
  'as',
  'excuse',
  'not',
  'to',
  'reuse',
  'plastic',
  'bag',
  'https',
  ':',
  '//t.co/thf8gunfpe',
  '#',
  'coronavirus',
  '#',
  'nashville'],
 ['#',
  'indiafightscorona',
  ':',
  'we',
  'have',
  '1524',
  '#',
  'covid',
  'testing',
  'laboratories',
  'in',
  'india',
  'and',
  'as',
 

In [ ]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, min_count=3, vector_size=300, window=5) #, iter=15

CPU times: user 3.68 s, sys: 34.2 ms, total: 3.72 s
Wall time: 3.39 s


In [ ]:
# model_tweets.wv.most_similar('france')
model_tweets.wv.most_similar('vaccine')

[('virus', 0.98159259557724),
 ('cure', 0.9626182913780212),
 ('fraud', 0.9538935422897339),
 ('labeled', 0.9533505439758301),
 ('arresting', 0.9519674777984619),
 ('alive', 0.9485616683959961),
 ('patent', 0.9483221769332886),
 ('_the', 0.9473593831062317),
 ('novel', 0.9443000555038452),
 ('administered', 0.9436068534851074)]

In [ ]:
model_tweets.init_sims()

<ipython-input-11-f1a8336e4a1e>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [ ]:
print(model_tweets)

Word2Vec<vocab=5266, vector_size=300, alpha=0.025>


In [ ]:
import numpy as np

In [ ]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:10<00:00, 620.23it/s]


In [ ]:
# features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.86      0.90      0.88      1013
        real       0.90      0.87      0.88      1106

    accuracy                           0.88      2119
   macro avg       0.88      0.88      0.88      2119
weighted avg       0.88      0.88      0.88      2119



###  Что будет, если использовать самый наивный метод?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()

In [ ]:
bow = vec.fit_transform(df.tweet)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.90      0.93      0.92       999
        real       0.94      0.91      0.92      1120

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Конечно, мы всегда можем поиграться с предобработкой.

###1

In [ ]:
tweet_clean = df.tweet.copy()

In [ ]:
# убрать ссылки # работает хуже (см.ниже)

# import re

# def remove_url(text):
#     url_pattern = re.compile(r'https?://\S+|t\.\S+')
#     return url_pattern.sub(r'',text)

# tweet_clean = df.tweet.apply(remove_url)
# tweet_clean[10]

# работает хуже:
#               precision    recall  f1-score   support

#         fake       0.84      0.91      0.87      1009
#         real       0.91      0.84      0.87      1110

#     accuracy                           0.87      2119
#    macro avg       0.87      0.87      0.87      2119
# weighted avg       0.87      0.87      0.87      2119

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(tweet_clean)]

100%|██████████| 6420/6420 [00:12<00:00, 525.90it/s]


In [ ]:
# sents_sw = sentences

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# stopwords.words("english")

# for token in tokens:
#     if token not in stop_words:
#         filtered_tokens.append(token)


sents_sw = []
for sent in tqdm(sentences):
    sent_sw = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        if word not in stopwords.words('english'):
            sent_sw.append( word )
    sents_sw.append( sent_sw )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_sw[10]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 6420/6420 [02:00<00:00, 53.48it/s]


['take',
 'simple',
 'daily',
 'precautions',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illnesses',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':',
 'https',
 ':',
 '//t.co/uargztrh5l',
 '.',
 'https',
 ':',
 '//t.co/biztxtukyk']

In [ ]:
from nltk.stem import WordNetLemmatizer
# import nltk
nltk.download('wordnet')

wnl = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# sents_lem = sents_sw

In [ ]:

sents_lem = []
for sent in sents_sw:
    sent_lem = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        sent_lem.append( wnl.lemmatize(word) )
    sents_lem.append( sent_lem )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_lem[10]

# sent = sentences[10]
# print(sent)
# sent_lem = []
# for word in sent:
#     # word_lem = wnl.lemmatize(word)
#     sent_lem.append( wnl.lemmatize(word) )
# # sents_lem.append( sent_lem )
# print(sent_lem)


['take',
 'simple',
 'daily',
 'precaution',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illness',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':',
 'http',
 ':',
 '//t.co/uargztrh5l',
 '.',
 'http',
 ':',
 '//t.co/biztxtukyk']

In [ ]:
from gensim.models.word2vec import Word2Vec
%time model_tweets_lem = Word2Vec(sents_lem, workers=4, min_count=3, vector_size=300, window=5) #, iter=15

CPU times: user 2.41 s, sys: 37.8 ms, total: 2.45 s
Wall time: 1.43 s


In [ ]:
print(model_tweets_lem)

Word2Vec<vocab=4769, vector_size=300, alpha=0.025>


In [ ]:
import numpy as np

In [ ]:
def get_lem_tok_sentence_embedding(sent_tok_lem):
    result = []
    for word in sent_tok_lem:
        if word in model_tweets_lem.wv:
            result.append(model_tweets_lem.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features_lem = [get_lem_tok_sentence_embedding(sentence) for sentence in tqdm(sents_lem)]

100%|██████████| 6420/6420 [00:01<00:00, 3987.61it/s]


In [ ]:
# features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_lem, df.label, test_size=0.33)#, random_state=0)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.88      0.90      0.89      1031
        real       0.90      0.89      0.89      1088

    accuracy                           0.89      2119
   macro avg       0.89      0.89      0.89      2119
weighted avg       0.89      0.89      0.89      2119



###2

In [ ]:
# tweet_clean = df.tweet.copy()

In [ ]:
# убрать ссылки

import re

def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|t\.\S+')
    return url_pattern.sub(r'',text)

tweet_clean = df.tweet.apply(remove_url)
tweet_clean[10]


'Take simple daily precautions to help prevent the spread of respiratory illnesses like #COVID19. Learn how to protect yourself from coronavirus (COVID-19):  '

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()

In [ ]:
bow = vec.fit_transform(tweet_clean)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.94      0.93       998
        real       0.95      0.92      0.94      1121

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



### PyTorch + LSTM

In [ ]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для макксимальной длины предложений.

In [ ]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [ ]:
max_len

1592

Это слишком много. Но какая длина обычно?

In [ ]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [ ]:
fd.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

Зададим максимум 200.

Возьмём те же w2v эмбеддинги.

In [ ]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [ ]:
features = [get_word_embedding(text, 200) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:20<00:00, 318.09it/s]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
len(features[0][0])

300

In [ ]:
len(X_train)

4301

In [ ]:
len(X_train[0])

200

In [ ]:
len(X_train[0][0])

300

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

torch.Size([4301, 200, 300])

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [13:52<00:00,  3.10s/it]

tensor(0.6855, grad_fn=<BinaryCrossEntropyBackward0>)


Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [ ]:
result = (output > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.5200566304860783

Но такую модель надо учить дольше(

###3
PyTorch + LSTM

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2023-08-31 05:55:17--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2023-08-31 05:55:18 (21.9 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
labels = (df.label == 'real').astype(int).to_list()

In [ ]:
# tweet_clean = df.tweet.copy()

In [ ]:
# убрать ссылки

import re

def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|t\.\S+')
    return url_pattern.sub(r'',text)

tweet_clean = df.tweet.apply(remove_url)
tweet_clean[10]


'Take simple daily precautions to help prevent the spread of respiratory illnesses like #COVID19. Learn how to protect yourself from coronavirus (COVID-19):  '

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(tweet_clean)]

100%|██████████| 6420/6420 [00:06<00:00, 930.43it/s]


In [ ]:
# sents_sw = sentences

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# stopwords.words("english")

# for token in tokens:
#     if token not in stop_words:
#         filtered_tokens.append(token)


sents_sw = []
for sent in tqdm(sentences):
    sent_sw = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        if word not in stopwords.words('english'):
            sent_sw.append( word )
    sents_sw.append( sent_sw )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_sw[10]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 6420/6420 [00:55<00:00, 115.29it/s]


['take',
 'simple',
 'daily',
 'precautions',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illnesses',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':']

In [ ]:
from nltk.stem import WordNetLemmatizer
# import nltk
nltk.download('wordnet')

wnl = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# sents_lem = sents_sw

In [ ]:

sents_lem = []
for sent in sents_sw:
    sent_lem = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        sent_lem.append( wnl.lemmatize(word) )
    sents_lem.append( sent_lem )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_lem[10]

# sent = sentences[10]
# print(sent)
# sent_lem = []
# for word in sent:
#     # word_lem = wnl.lemmatize(word)
#     sent_lem.append( wnl.lemmatize(word) )
# # sents_lem.append( sent_lem )
# print(sent_lem)


['take',
 'simple',
 'daily',
 'precaution',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illness',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':']

Нужно заранее задать размер для макксимальной длины предложений.

In [ ]:
# token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(sents_lem, key=len))

In [ ]:
max_len

1012

Это слишком много. Но какая длина обычно?

In [ ]:
from collections import Counter
fd = Counter([len(tokens) for tokens in sents_lem])

In [ ]:
fd.most_common(10)

[(12, 281),
 (11, 271),
 (13, 270),
 (14, 256),
 (15, 243),
 (9, 232),
 (10, 229),
 (16, 216),
 (8, 214),
 (17, 212)]

####Зададим максимум 50 #200.

In [ ]:
from gensim.models.word2vec import Word2Vec
%time model_tweets_lem = Word2Vec(sents_lem, workers=4, min_count=3, vector_size=300, window=5) #, iter=15

CPU times: user 2.6 s, sys: 47.9 ms, total: 2.65 s
Wall time: 1.58 s


In [ ]:
print(model_tweets_lem)

Word2Vec<vocab=4717, vector_size=300, alpha=0.025>


In [ ]:
import numpy as np

In [ ]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets_lem.wv:
                result.append(model_tweets_lem.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [ ]:
features = [get_word_embedding(text, 50) for text in tqdm(sents_lem)]

100%|██████████| 6420/6420 [00:04<00:00, 1443.50it/s]


In [ ]:
from sklearn.model_selection import train_test_split

In [243]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, stratify=labels)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
len(features[0][0])

300

In [ ]:
len(X_train)

4301

In [ ]:
len(X_train[0])

50

In [ ]:
len(X_train[0][0])

300

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

torch.Size([4301, 50, 300])

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=512):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
train_one_epoch(in_data, targets)

100%|██████████| 9/9 [00:15<00:00,  1.67s/it]

tensor(0.6912, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
for i in range(15):
    train_one_epoch(in_data, targets)


100%|██████████| 9/9 [00:19<00:00,  2.16s/it]


tensor(0.6905, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:19<00:00,  2.13s/it]


tensor(0.6905, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:17<00:00,  1.99s/it]


...

tensor(0.6903, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:18<00:00,  2.02s/it]


tensor(0.6903, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:15<00:00,  1.73s/it]

tensor(0.6903, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
with torch.no_grad():
    print( (( net(in_data).reshape(-1) ) > 0.5).sum() )

tensor(4301)


Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [ ]:
result = (output > 0.5) == targets_test

In [ ]:
(output > 0.5).sum()

tensor(2118)

In [ ]:
targets_test.sum()

tensor(1118.)

In [ ]:
result.sum().item() / len(result)

0.5280792826805096

In [ ]:
output.tolist()

[0.5202255845069885,
 0.5202255845069885,
 0.5202259421348572,
 0.5202257633209229,
 0.5202256441116333,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202256441116333,
 0.5202255845069885,
 0.5202255845069885,
 0.5202260613441467,
 0.5202255845069885,
 0.5532165765762329,
 0.5202255845069885,
 0.5202257633209229,
 0.5202264189720154,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5207542777061462,
 0.5202286839485168,
 0.5202255845069885,
 0.5202255845069885,
 0.520226001739502,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202326774597168,
 0.5207731127738953,
 0.5202257633209229,
 0.5202255845069885,
 0.5202255845069885,
 0.5202258229255676,
 0.5202261209487915,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202255845069885,
 0.5202321410179138,
 0.5202255845069885,
 0.52022558450

In [ ]:
# targets_test.tolist()

In [ ]:
result.sum().item() / (output > 0.5).sum().item()

0.5283286118980169

In [ ]:
predicted = (output > 0.5).tolist()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1001
           1       0.53      1.00      0.69      1118

    accuracy                           0.53      2119
   macro avg       0.76      0.50      0.35      2119
weighted avg       0.75      0.53      0.37      2119



и такую модель не надо учить дольше

###4 Net3

In [353]:
# net2 = net

In [260]:
class Net3(nn.Module):

    def __init__(self):
        super(Net3, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.lin1 = nn.Linear(100, 100)
        self.lin2 = nn.Linear(100, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net3()
print(net)

Net3(
  (lstm): LSTM(300, 100)
  (lin1): Linear(in_features=100, out_features=100, bias=True)
  (lin2): Linear(in_features=100, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [299]:
net3 = net

In [245]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [261]:
in_data.shape

torch.Size([4301, 50, 300])

In [262]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [263]:
def train_one_epoch(in_data, targets, batch_size=512):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [264]:
train_one_epoch(in_data, targets)

100%|██████████| 9/9 [00:16<00:00,  1.85s/it]

tensor(0.6942, grad_fn=<BinaryCrossEntropyBackward0>)


In [285]:
for i in range(25):
    train_one_epoch(in_data, targets)


100%|██████████| 9/9 [00:16<00:00,  1.85s/it]


tensor(0.6925, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:15<00:00,  1.77s/it]


tensor(0.6925, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:15<00:00,  1.69s/it]


...

tensor(0.6924, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:16<00:00,  1.81s/it]


tensor(0.6924, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:15<00:00,  1.77s/it]

tensor(0.6924, grad_fn=<BinaryCrossEntropyBackward0>)


In [286]:
with torch.no_grad():
    print( (( net(in_data).reshape(-1) ) > 0.5).sum() )

tensor(4301)


Что получилось?

In [287]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [288]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [289]:
result = (output > 0.5) == targets_test

In [290]:
(output > 0.5).sum()

tensor(2119)

In [291]:
targets_test.sum()

tensor(1109.)

In [292]:
result.sum().item() / len(result)

0.5233600755073148

In [293]:
output.tolist()

[0.5208700895309448,
 0.5208715796470642,
 0.5208715796470642,
 0.5208715796470642,
 0.5208710432052612,
 0.5208715796470642,
 0.5208715796470642,
 0.520869255065918,
 0.5208715796470642,
 0.5208522081375122,
 0.5208715796470642,
 0.5209370851516724,
 0.5208603143692017,
 0.5208756327629089,
 0.5208715200424194,
 0.5208715796470642,
 0.5208715200424194,
 0.5208752155303955,
 0.5208707451820374,
 0.5208714008331299,
 0.5208715796470642,
 0.5208715796470642,
 0.5208715796470642,
 0.5208715796470642,
 0.5208706259727478,
 0.5208715796470642,
 0.5208715796470642,
 0.5208714604377747,
 0.5208675861358643,
 0.5208722352981567,
 0.5208715796470642,
 0.5208715796470642,
 0.5208714604377747,
 0.5208715796470642,
 0.5208712816238403,
 0.5208680629730225,
 0.5208715796470642,
 0.5208715796470642,
 0.5208715796470642,
 0.5208696722984314,
 0.5208712220191956,
 0.5208715796470642,
 0.5208715796470642,
 0.5208710432052612,
 0.5208715796470642,
 0.5209006667137146,
 0.5208665132522583,
 0.52087157964

In [294]:
# targets_test.tolist()

In [295]:
result.sum().item() / (output > 0.5).sum().item()

0.5233600755073148

In [296]:
predicted = (output > 0.5).tolist()

In [297]:
from sklearn.metrics import classification_report

In [298]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1010
           1       0.52      1.00      0.69      1109

    accuracy                           0.52      2119
   macro avg       0.26      0.50      0.34      2119
weighted avg       0.27      0.52      0.36      2119



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Но такую модель бесполезно учить дольше(

###5 NAdam

In [300]:
class Net4(nn.Module):

    def __init__(self):
        super(Net4, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.lin1 = nn.Linear(100, 100)
        self.lin2 = nn.Linear(100, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net4()
net4 = net
print(net)

Net4(
  (lstm): LSTM(300, 100)
  (lin1): Linear(in_features=100, out_features=100, bias=True)
  (lin2): Linear(in_features=100, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [349]:
net4 = net

In [301]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [302]:
in_data.shape

torch.Size([4301, 50, 300])

In [303]:
optimizer = optim.NAdam(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [304]:
def train_one_epoch(in_data, targets, batch_size=512):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [305]:
train_one_epoch(in_data, targets)

100%|██████████| 9/9 [00:22<00:00,  2.45s/it]

tensor(0.6880, grad_fn=<BinaryCrossEntropyBackward0>)


In [319]:
for i in range(25):
    train_one_epoch(in_data, targets)


100%|██████████| 9/9 [00:16<00:00,  1.81s/it]


tensor(0.6852, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:16<00:00,  1.83s/it]


tensor(0.6347, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:13<00:00,  1.51s/it]


...

tensor(0.2514, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:16<00:00,  1.78s/it]


tensor(0.2597, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:16<00:00,  1.84s/it]

tensor(0.2352, grad_fn=<BinaryCrossEntropyBackward0>)


In [320]:
with torch.no_grad():
    print( (( net(in_data).reshape(-1) ) > 0.5).sum() )

tensor(2211)


Что получилось?

In [321]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [322]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [323]:
result = (output > 0.5) == targets_test

In [324]:
(output > 0.5).sum()

tensor(1118)

In [325]:
targets_test.sum()

tensor(1109.)

In [326]:
result.sum().item() / len(result)

0.8947616800377537

In [327]:
output.tolist()

[0.7761735916137695,
 0.11186707764863968,
 0.9889931082725525,
 0.0068610734306275845,
 0.9887531995773315,
 0.9766091704368591,
 0.6843317151069641,
 0.9954311847686768,
 0.8231070041656494,
 0.999731719493866,
 0.06785958260297775,
 0.9999090433120728,
 0.2617702782154083,
 0.9995651841163635,
 0.9483820199966431,
 0.9779883623123169,
 0.7498793601989746,
 0.9593414068222046,
 0.7251865863800049,
 0.9967532753944397,
 0.1217525452375412,
 0.47954922914505005,
 0.9520097374916077,
 0.3872832953929901,
 0.8926473259925842,
 0.07130485773086548,
 0.11616623401641846,
 0.9236013293266296,
 0.9856098294258118,
 0.9984536170959473,
 0.059610575437545776,
 0.09987325221300125,
 0.9745621681213379,
 0.35568878054618835,
 0.9955173134803772,
 0.9953403472900391,
 0.11259972304105759,
 0.9933598637580872,
 0.08376277983188629,
 0.3638722896575928,
 0.7785667777061462,
 0.7101771235466003,
 0.9815778136253357,
 0.9118125438690186,
 0.1389591246843338,
 0.9998540878295898,
 0.9811995029449463,


In [328]:
# targets_test.tolist()

In [329]:
result.sum().item() / (output > 0.5).sum().item()

1.6958855098389982

In [330]:
predicted = (output > 0.5).tolist()

In [331]:
from sklearn.metrics import classification_report

In [332]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1010
           1       0.90      0.90      0.90      1109

    accuracy                           0.89      2119
   macro avg       0.89      0.89      0.89      2119
weighted avg       0.89      0.89      0.89      2119



Вот такую модель стоит поучить подольше)

###6 50 epochs

In [333]:
for i in range(25):
    train_one_epoch(in_data, targets)


100%|██████████| 9/9 [00:25<00:00,  2.84s/it]


tensor(0.2258, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:17<00:00,  1.95s/it]


tensor(0.2181, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:19<00:00,  2.22s/it]


...

tensor(0.1269, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:16<00:00,  1.82s/it]


tensor(0.1188, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:15<00:00,  1.72s/it]

tensor(0.1829, grad_fn=<BinaryCrossEntropyBackward0>)


In [334]:
with torch.no_grad():
    print( (( net(in_data).reshape(-1) ) > 0.5).sum() )

tensor(2270)


Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [335]:
with torch.no_grad():
    output2 = net(in_data_test).reshape(-1)

In [336]:
# result = (output > 0.5) == targets_test

In [337]:
(output2 > 0.5).sum()

tensor(1136)

In [338]:
# targets_test.sum()

In [339]:
# result.sum().item() / len(result)

In [340]:
# output.tolist()

In [341]:
# targets_test.tolist()

In [342]:
# result.sum().item() / (output > 0.5).sum().item()

In [343]:
predicted2 = (output2 > 0.5).tolist()

In [344]:
from sklearn.metrics import classification_report

In [345]:
print(classification_report(y_test, predicted2))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      1010
           1       0.91      0.94      0.93      1109

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



И такую модель можно учить подольше)

###7 PyTorch + LSTM (NAdam)

In [355]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2023-08-31 10:52:50--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.2’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2023-08-31 10:52:50 (21.8 MB/s) - ‘Constraint_Train.csv.2’ saved [1253562/1253562]



In [356]:
import pandas as pd

In [357]:
df = pd.read_csv('Constraint_Train.csv')

In [358]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [359]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [360]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [361]:
labels = (df.label == 'real').astype(int).to_list()

In [362]:
# tweet_clean = df.tweet.copy()

In [363]:
# убрать ссылки

import re

def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|t\.\S+')
    return url_pattern.sub(r'',text)

tweet_clean = df.tweet.apply(remove_url)
tweet_clean[10]


'Take simple daily precautions to help prevent the spread of respiratory illnesses like #COVID19. Learn how to protect yourself from coronavirus (COVID-19):  '

In [364]:
sentences = [word_tokenize(text.lower()) for text in tqdm(tweet_clean)]

100%|██████████| 6420/6420 [00:13<00:00, 475.43it/s]


In [365]:
# sents_sw = sentences

In [366]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# stopwords.words("english")

# for token in tokens:
#     if token not in stop_words:
#         filtered_tokens.append(token)


sents_sw = []
for sent in tqdm(sentences):
    sent_sw = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        if word not in stopwords.words('english'):
            sent_sw.append( word )
    sents_sw.append( sent_sw )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_sw[10]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 6420/6420 [01:40<00:00, 63.87it/s]


['take',
 'simple',
 'daily',
 'precautions',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illnesses',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':']

In [367]:
from nltk.stem import WordNetLemmatizer
# import nltk
nltk.download('wordnet')

wnl = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [368]:
# sents_lem = sents_sw

In [369]:

sents_lem = []
for sent in sents_sw:
    sent_lem = []
    for word in sent:
        # word_lem = wnl.lemmatize(word)
        sent_lem.append( wnl.lemmatize(word) )
    sents_lem.append( sent_lem )

# sents_lem = [[wnl.lemmatize(word) for word in sentence] for sentence in sentences]
sents_lem[10]

# sent = sentences[10]
# print(sent)
# sent_lem = []
# for word in sent:
#     # word_lem = wnl.lemmatize(word)
#     sent_lem.append( wnl.lemmatize(word) )
# # sents_lem.append( sent_lem )
# print(sent_lem)


['take',
 'simple',
 'daily',
 'precaution',
 'help',
 'prevent',
 'spread',
 'respiratory',
 'illness',
 'like',
 '#',
 'covid19',
 '.',
 'learn',
 'protect',
 'coronavirus',
 '(',
 'covid-19',
 ')',
 ':']

Нужно заранее задать размер для макксимальной длины предложений.

In [370]:
# token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(sents_lem, key=len))

In [371]:
max_len

1012

Это слишком много. Но какая длина обычно?

In [372]:
from collections import Counter
fd = Counter([len(tokens) for tokens in sents_lem])

In [373]:
fd.most_common(10)

[(12, 281),
 (11, 271),
 (13, 270),
 (14, 256),
 (15, 243),
 (9, 232),
 (10, 229),
 (16, 216),
 (8, 214),
 (17, 212)]

####Зададим максимум 50 #200.

In [374]:
from gensim.models.word2vec import Word2Vec
%time model_tweets_lem = Word2Vec(sents_lem, workers=4, min_count=3, vector_size=300, window=5) #, iter=15

CPU times: user 2.73 s, sys: 29.3 ms, total: 2.76 s
Wall time: 1.64 s


In [375]:
print(model_tweets_lem)

Word2Vec<vocab=4717, vector_size=300, alpha=0.025>


In [376]:
import numpy as np

In [377]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets_lem.wv:
                result.append(model_tweets_lem.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [378]:
features = [get_word_embedding(text, 50) for text in tqdm(sents_lem)]

100%|██████████| 6420/6420 [00:07<00:00, 856.18it/s]


In [379]:
from sklearn.model_selection import train_test_split

In [380]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, stratify=labels)

In [381]:
import torch
import torch.nn as nn
import torch.optim as optim

In [382]:
len(features[0][0])

300

In [383]:
len(X_train)

4301

In [384]:
len(X_train[0])

50

In [385]:
len(X_train[0][0])

300

In [386]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [387]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [388]:
in_data.shape

torch.Size([4301, 50, 300])

In [389]:
optimizer = optim.NAdam(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [390]:
def train_one_epoch(in_data, targets, batch_size=512):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [391]:
train_one_epoch(in_data, targets)

100%|██████████| 9/9 [00:20<00:00,  2.28s/it]

tensor(0.6820, grad_fn=<BinaryCrossEntropyBackward0>)


In [392]:
for i in range(15):
    train_one_epoch(in_data, targets)


100%|██████████| 9/9 [00:17<00:00,  1.91s/it]


tensor(1.2614, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:17<00:00,  1.90s/it]


tensor(0.6838, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:14<00:00,  1.57s/it]


...

tensor(0.4138, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:19<00:00,  2.11s/it]


tensor(0.3378, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 9/9 [00:17<00:00,  1.94s/it]

tensor(0.3317, grad_fn=<BinaryCrossEntropyBackward0>)


In [393]:
with torch.no_grad():
    print( (( net(in_data).reshape(-1) ) > 0.5).sum() )

tensor(2200)


Что получилось?

In [394]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [395]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [396]:
result = (output > 0.5) == targets_test

In [397]:
(output > 0.5).sum()

tensor(1040)

In [398]:
targets_test.sum()

tensor(1109.)

In [399]:
result.sum().item() / len(result)

0.8702218027371401

In [400]:
output.tolist()

[0.11248127371072769,
 0.05571185424923897,
 0.24447505176067352,
 0.5725192427635193,
 0.36646580696105957,
 0.5996150374412537,
 0.23997104167938232,
 0.6209104061126709,
 0.04432802274823189,
 0.053202077746391296,
 0.3215315639972687,
 0.9991831183433533,
 0.7977806925773621,
 0.18543130159378052,
 0.5169302821159363,
 0.25031906366348267,
 0.086492620408535,
 0.913255512714386,
 0.06001150608062744,
 0.2417905032634735,
 0.9962970614433289,
 0.1766003966331482,
 0.18511372804641724,
 0.9961565136909485,
 0.08994948118925095,
 0.9992733597755432,
 0.25037407875061035,
 0.32007405161857605,
 0.9969260096549988,
 0.9950934648513794,
 0.23531420528888702,
 0.05772855132818222,
 0.6132725477218628,
 0.5348225235939026,
 0.9776797890663147,
 0.04962654411792755,
 0.23643627762794495,
 0.316588819026947,
 0.055427249521017075,
 0.72323077917099,
 0.9848832488059998,
 0.98775315284729,
 0.17571720480918884,
 0.029595933854579926,
 0.26915186643600464,
 0.7682545781135559,
 0.8064565062522

In [401]:
# targets_test.tolist()

In [402]:
result.sum().item() / (output > 0.5).sum().item()

1.773076923076923

In [403]:
predicted = (output > 0.5).tolist()

In [404]:
from sklearn.metrics import classification_report

In [405]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1010
           1       0.90      0.84      0.87      1109

    accuracy                           0.87      2119
   macro avg       0.87      0.87      0.87      2119
weighted avg       0.87      0.87      0.87      2119



Но такую модель надо учить дольше(